In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('dataset/data.csv',encoding='ISO-8859-1')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
# dropping all rows that have na values

data = data.dropna()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
# dropping InvoiceNo and StockCode columns are irrelevant
data.drop(['InvoiceNo','StockCode'],axis=1,inplace=True)
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
# regular expression to remove punctuation and lowers capital letters
import re
def clean(description):
    return re.sub(r'[^\w\s]','',description).lower()

In [6]:
clean('HI.!!!')

'hi'

In [7]:
# using the apply function
data['Description'] = data['Description'].apply(clean)
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,white hanging heart tlight holder,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,red woolly hottie white heart,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [8]:
# reset the index
data = data.reset_index().drop('index',axis=1)

In [9]:
# total 
data['Total'] = data['UnitPrice'] * data['Quantity']

In [10]:
# Vectorise the description
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')

In [11]:
descriptions = vectorizer.fit_transform(data['Description'])
descriptions

<406829x2093 sparse matrix of type '<class 'numpy.int64'>'
	with 1640936 stored elements in Compressed Sparse Row format>

In [12]:
# Cluster the Products & Create Features
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)
kmeans.fit(descriptions)

KMeans(n_clusters=5)

In [13]:
# assign through the kmeans
data['Product'] = kmeans.labels_
data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total,Product
0,white hanging heart tlight holder,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30,4
1,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,1
2,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00,1
3,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,1
4,red woolly hottie white heart,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34,4


In [14]:
# Create a new dataframe called customers
customers = pd.DataFrame({'CustomerID':data['CustomerID'].unique()})
customers.head()

,CustomerID
0,17850.0
1,13047.0
2,12583.0
3,13748.0
4,15100.0


In [15]:
# create 5 columns, one for each product category and set to 0
customers['Product0'] = 0
customers['Product1'] = 0
customers['Product2'] = 0
customers['Product3'] = 0
customers['Product4'] = 0
customers.head()

,CustomerID,Product0,Product1,Product2,Product3,Product4
0,17850.0,0,0,0,0,0
1,13047.0,0,0,0,0,0
2,12583.0,0,0,0,0,0
3,13748.0,0,0,0,0,0
4,15100.0,0,0,0,0,0


In [16]:
from tqdm import tqdm
# For each index in the data
for index in tqdm(range(len(data))):
    # finds the customer ID of that row’s customer ID
    customerID = data.loc[index,'CustomerID']
    # finds the product category of whatever product was in that row
    product_category = data.loc[index,'Product']
    # finds the amount spent on that item
    product_price = data.loc[index,'Total']
    # ine finds the amount spent on that item
    customers_table_index = customers[customers['CustomerID']==customerID].index[0]
    # The row whose customer ID matches the one we want to target’s product column has the product price added to it.
    customers.loc[customers_table_index, 'Product'+str(product_category)] += product_price

100%|██████████| 406829/406829 [09:06<00:00, 744.80it/s]


In [17]:
customers

,CustomerID,Product0,Product1,Product2,Product3,Product4
0,17850.0,0.00,4199.87,99.0,321.30,668.46
1,13047.0,33.00,2366.66,0.0,518.38,161.06
2,12583.0,1136.42,4499.98,177.6,1373.34,0.00
3,13748.0,0.00,861.87,0.0,52.08,34.30
4,15100.0,0.00,635.10,0.0,0.00,0.00
...,...,...,...,...,...,...
4367,13436.0,0.00,196.89,0.0,0.00,0.00
4368,15520.0,64.80,278.70,0.0,0.00,0.00
4369,13298.0,0.00,360.00,0.0,0.00,0.00
4370,14569.0,0.00,203.59,0.0,13.85,9.95


In [18]:
# Creating Purchase History Descriptions
customers['PurchaseCount'] = 0
customers['PurchaseMin'] = 0
customers['PurchaseMax'] = 0
customers['PurchaseMean'] = 0
customers['PurchaseSum'] = 0
customers.head()

,CustomerID,Product0,Product1,Product2,Product3,Product4,PurchaseCount,PurchaseMin,PurchaseMax,PurchaseMean,PurchaseSum
0,17850.0,0.00,4199.87,99.0,321.30,668.46,0,0,0,0,0
1,13047.0,33.00,2366.66,0.0,518.38,161.06,0,0,0,0,0
2,12583.0,1136.42,4499.98,177.6,1373.34,0.00,0,0,0,0,0
3,13748.0,0.00,861.87,0.0,52.08,34.30,0,0,0,0,0
4,15100.0,0.00,635.10,0.0,0.00,0.00,0,0,0,0,0


In [19]:
from tqdm import tqdm
# iterates over all the rows in customers. Since each row is a unique customer, we don’t have to add a .unique()
for index in tqdm(range(len(customers))):
    # stores the Customer ID of a certain row into customerID
    customerID = customers.loc[index,'CustomerID']
    # selects the Total column, which can be thought of as the ‘purchase record’ for that customer
    history = data[data['CustomerID']==customerID]['Total']
    # these five lines simply fills in a row’s history attributes with attributes from history
    customers.loc[index,'PurchaseCount'] = history.count()
    customers.loc[index,'PurchaseMin'] = history.min()
    customers.loc[index,'PurchaseMax'] = history.max()
    customers.loc[index,'PurchaseMean'] = history.mean()
    customers.loc[index,'PurchaseSum'] = history.sum()

100%|██████████| 4372/4372 [00:38<00:00, 112.70it/s]


In [20]:
customers

,CustomerID,Product0,Product1,Product2,Product3,Product4,PurchaseCount,PurchaseMin,PurchaseMax,PurchaseMean,PurchaseSum
0,17850.0,0.00,4199.87,99.0,321.30,668.46,312,-30.60,107.25,16.950737,5288.63
1,13047.0,33.00,2366.66,0.0,518.38,161.06,196,-15.00,68.00,15.709694,3079.10
2,12583.0,1136.42,4499.98,177.6,1373.34,0.00,251,-60.84,132.80,28.634821,7187.34
3,13748.0,0.00,861.87,0.0,52.08,34.30,28,9.36,204.00,33.866071,948.25
4,15100.0,0.00,635.10,0.0,0.00,0.00,6,-131.40,350.40,105.850000,635.10
...,...,...,...,...,...,...,...,...,...,...,...
4367,13436.0,0.00,196.89,0.0,0.00,0.00,12,10.20,25.50,16.407500,196.89
4368,15520.0,64.80,278.70,0.0,0.00,0.00,18,9.48,34.68,19.083333,343.50
4369,13298.0,0.00,360.00,0.0,0.00,0.00,2,90.00,270.00,180.000000,360.00
4370,14569.0,0.00,203.59,0.0,13.85,9.95,12,1.65,122.40,18.949167,227.39


In [21]:
# By default, Foreign will be 0, because the majority of sales are from the United Kingdom.
customers['Foreign'] = 0

In [22]:
for index in tqdm(range(len(customers))):
    # retrieves the customer ID to query the data DataFrame.
    customerID = customers.loc[index,'CustomerID']
    # selects the rows whose CustomerID column is equal to the assigned customerID
    # because this data subset was taken from data
    # its original indices are maintained, meaning that the first row probably won’t be of index 0.
    # To solve this issue, we can use .reset_index(), which ensures that the first row has index 0
    customerCountry = data[data['CustomerID']==customerID] .reset_index().loc[0,'Country']
    # if it is not equal to ‘United Kingdom’, then we will assign the ‘Foreign’ column in that cell to 1
    if customerCountry == 'United Kingdom':
        customers.loc[index,'Foreign'] = 1

100%|██████████| 4372/4372 [00:20<00:00, 217.59it/s]


In [23]:
customers

,CustomerID,Product0,Product1,Product2,Product3,Product4,PurchaseCount,PurchaseMin,PurchaseMax,PurchaseMean,PurchaseSum,Foreign
0,17850.0,0.00,4199.87,99.0,321.30,668.46,312,-30.60,107.25,16.950737,5288.63,1
1,13047.0,33.00,2366.66,0.0,518.38,161.06,196,-15.00,68.00,15.709694,3079.10,1
2,12583.0,1136.42,4499.98,177.6,1373.34,0.00,251,-60.84,132.80,28.634821,7187.34,0
3,13748.0,0.00,861.87,0.0,52.08,34.30,28,9.36,204.00,33.866071,948.25,1
4,15100.0,0.00,635.10,0.0,0.00,0.00,6,-131.40,350.40,105.850000,635.10,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4367,13436.0,0.00,196.89,0.0,0.00,0.00,12,10.20,25.50,16.407500,196.89,1
4368,15520.0,64.80,278.70,0.0,0.00,0.00,18,9.48,34.68,19.083333,343.50,1
4369,13298.0,0.00,360.00,0.0,0.00,0.00,2,90.00,270.00,180.000000,360.00,1
4370,14569.0,0.00,203.59,0.0,13.85,9.95,12,1.65,122.40,18.949167,227.39,1


Converted user/product centric data into data purely ordered by a customer while retaining important information and setting up the stage to make K Means and analysis easy